In [1]:
import pandas as pd

In [3]:
df=pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [7]:
df=df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18285 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18285 non-null  int64 
 1   title   18285 non-null  object
 2   author  18285 non-null  object
 3   text    18285 non-null  object
 4   label   18285 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 857.1+ KB


In [9]:
X=df.drop(columns='label')
y=df['label']

In [17]:
from keras.layers import Embedding,LSTM,Dense
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [78]:
voc_size=10000

In [21]:
messages=X.copy()

In [25]:
messages.reset_index(inplace=True)

In [47]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [57]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nihat_mhvorou\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nihat_mhvorou\AppData\Roaming\nltk_data...


True

In [176]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [61]:
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)


In [63]:
corpus

['house dem aide even see comey letter jason chaffetz tweeted',
 'flynn hillary clinton big woman campus breitbart',
 'truth might get fired',
 'civilian killed single u airstrike identified',
 'iranian woman jailed fictional unpublished story woman stoned death adultery',
 'jackie mason hollywood would love trump bombed north korea lack trans bathroom exclusive video breitbart',
 'beno hamon win french socialist party presidential nomination new york time',
 'back channel plan ukraine russia courtesy trump associate new york time',
 'obama organizing action partner soros linked indivisible disrupt trump agenda',
 'bbc comedy sketch real housewife isi cause outrage',
 'russian researcher discover secret nazi military base treasure hunter arctic photo',
 'u official see link trump russia',
 'yes paid government troll social medium blog forum website',
 'major league soccer argentine find home success new york time',
 'well fargo chief abruptly step new york time',
 'anonymous donor pay 

In [80]:
onehot_repr=[one_hot(word,voc_size) for word in corpus]
onehot_repr

[[5478, 6367, 9919, 4485, 2384, 1555, 9952, 636, 9233, 8407],
 [5314, 5858, 3347, 8385, 9313, 6992, 2486],
 [3147, 5487, 7608, 9246],
 [7327, 6624, 180, 3066, 7397, 8421],
 [3707, 9313, 3196, 6294, 6905, 2200, 9313, 608, 8248, 1059],
 [5605,
  9206,
  6594,
  8203,
  3265,
  8002,
  3646,
  7627,
  1561,
  2548,
  9086,
  2662,
  5086,
  1336,
  2486],
 [2157, 8356, 7749, 6972, 7849, 5164, 2142, 1761, 2293, 9690, 8363],
 [4046, 880, 8501, 5410, 6346, 2250, 8002, 3014, 2293, 9690, 8363],
 [8743, 5445, 4596, 6050, 1472, 1205, 2520, 4684, 8002, 2818],
 [5992, 2078, 4830, 869, 3588, 4827, 9007, 2859],
 [4958, 1115, 6233, 4527, 6158, 5482, 7111, 5687, 4266, 5307, 426],
 [3066, 2911, 2384, 6509, 8002, 6346],
 [6111, 9840, 5329, 6295, 1075, 6085, 4149, 6456, 2647],
 [7059, 7307, 456, 7249, 1400, 5926, 6738, 2293, 9690, 8363],
 [6868, 2657, 9463, 7516, 3848, 2293, 9690, 8363],
 [9863, 9724, 6047, 1448, 4573, 8736, 8677, 6518, 5229, 2486],
 [3570, 6046, 5858],
 [1298, 3054, 2309, 2910, 8002, 36

In [87]:
sent_length=20
embedding_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

In [89]:
embedding_docs

array([[   0,    0,    0, ...,  636, 9233, 8407],
       [   0,    0,    0, ..., 9313, 6992, 2486],
       [   0,    0,    0, ..., 5487, 7608, 9246],
       ...,
       [   0,    0,    0, ..., 2293, 9690, 8363],
       [   0,    0,    0, ..., 4462, 7294, 3594],
       [   0,    0,    0, ...,  724, 4531, 5148]])

In [142]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features))
model.build(input_shape=(None,sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 20, 40)         │       400,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 100)            │        56,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 456,501 (1.74 MB)

 Trainable params: 456,501 (1.74 MB)

 Non-trainable params: 0 (0.00 B)

In [144]:
len(embedding_docs),y.shape

(18285, (18285,))

In [146]:
import numpy as np
X_final=np.array(embedding_docs)
y_final=np.array(y)

In [148]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [150]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [152]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7885 - loss: 0.4208 - val_accuracy: 0.9145 - val_loss: 0.2001
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9456 - loss: 0.1389 - val_accuracy: 0.9234 - val_loss: 0.1947
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9774 - loss: 0.0722 - val_accuracy: 0.9188 - val_loss: 0.2517
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9872 - loss: 0.0414 - val_accuracy: 0.9112 - val_loss: 0.2889
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9925 - loss: 0.0245 - val_accuracy: 0.9153 - val_loss: 0.3638
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9968 - loss: 0.0130 - val_accuracy: 0.9135 - val_loss: 0.4253
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9984 - loss: 0.0065 - val_accuracy: 0.9123 - val_loss: 0.4556
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9993 - loss: 0.0034 - val_accuracy: 0.

In [153]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [154]:
from sklearn.metrics import confusion_matrix

In [155]:
confusion_matrix(y_test,y_pred)

array([[3126,  293],
       [ 279, 2337]], dtype=int64)

In [156]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9052195526097763

In [157]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features))
model.build(input_shape=(None,sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [158]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.7632 - loss: 0.4549 - val_accuracy: 0.9034 - val_loss: 0.2174
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9411 - loss: 0.1516 - val_accuracy: 0.9215 - val_loss: 0.1975
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9717 - loss: 0.0833 - val_accuracy: 0.9155 - val_loss: 0.2116
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9813 - loss: 0.0581 - val_accuracy: 0.9185 - val_loss: 0.3003
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9901 - loss: 0.0344 - val_accuracy: 0.9054 - val_loss: 0.2738
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9926 - loss: 0.0227 - val_accuracy: 0.9114 - val_loss: 0.3096
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9948 - loss: 0.0161 - val_accuracy: 0.9105 - val_loss: 0.3551
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9964 - loss: 0.0124 - val_accuracy: 0.

In [159]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [160]:
confusion_matrix(y_test,y_pred)

array([[3131,  288],
       [ 250, 2366]], dtype=int64)

In [161]:
accuracy_score(y_test,y_pred)

0.9108533554266777

In [204]:
df.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that...",0
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...,0


In [268]:
model.save('fakenewschecker.keras')